In [1]:
import ujson
import csv
import os
import shutil
import pathlib as pl

In [2]:
### Settings
HOME_PATH = "..\data\Acelerometro"              # Location of the working directory
DATASET_PATH = "..\data\Acelerometro"   # Upload your JSON samples to this directory
OUT_PATH = "..\data\Acelerometro\zip"           # Where output files go (will be deleted and recreated)
OUT_ZIP = "data\Acelerometro\zip\out.zip"        # Where to store the zipped output files

In [3]:
### Delete output directory (if it exists) and recreate it
if os.path.exists(OUT_PATH):
  shutil.rmtree(OUT_PATH)
os.makedirs(OUT_PATH)

In [4]:
from pathlib import Path

# specify the directory you want to list files from
data_directory = Path(DATASET_PATH)

# use the glob method to list all json files
json_files = list(data_directory.glob('*.json'))


In [5]:

### Read JSON files

# Go through each file in the input directory
for filename in json_files:

  # Read the JSON file
  header = ["timestamp"]
  with open(filename, 'r') as json_file:

    # Load JSON
    data = ujson.load(json_file)

    # Parse the sample rate, header, and data
    sample_rate = data['payload']['interval_ms']
    for heading in data['payload']['sensors']:
      header.append(heading['name'])
    values = data['payload']['values']
    header.append('label')
    label = filename.name.split('.')[0]

    # Write header to CSV file
    out_filepath = os.path.splitext(filename)[0] + ".csv"
    with open(out_filepath, 'w') as csv_file:
      writer = csv.writer(csv_file)
      writer.writerow(header)

      # Prepend timestamp and write data rows to CSV file
      for t, line in enumerate(values):
        timestamp = t * sample_rate
        line.insert(0, timestamp)
        line.insert(4, label)
        writer.writerow(line)